#### ***1-Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества***

select job_industry_category, count(customer_id) cnt from hw.customer group by job_industry_category order by cnt desc

https://drive.google.com/file/d/1E-gROll-4K-_0YvT5ZcffAuK1r4JutFw/view?usp=sharing

#### ***2-Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности***gory

select date_trunc('month', transaction_date::timestamp) mth, job_industry_category, sum(list_price::float) sm

from hw.transaction a left join hw.customer b on a.customer_id = b.customer_id 

where job_industry_category is not null group by mth, job_industry_category order by mth, job_industry_category

https://drive.google.com/file/d/1FVEm6F3KJqypOuCxffqfEdhAAi3vujeZ/view?usp=sharing

#### ***3-Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT***

select brand, count(transaction_id) from hw.transaction a left join hw.customer b on a.customer_id = b.customer_id

where online_order = true and order_status = 'Approved' and job_industry_category = 'IT' group by brand

https://drive.google.com/file/d/1QhGUX0AF-mxaVi7m0t6iPEiC125DjTEq/view?usp=sharing

#### ***4-Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат.***
select b.customer_id, sum(list_price) sm, max(list_price) mx, min(list_price) mn, count(list_price) cnt

from hw.transaction a left join hw.customer b on a.customer_id = b.customer_id

where b.customer_id is not null group by b.customer_id order by sm desc, cnt desc

https://drive.google.com/file/d/15RO9SG7_16IpgAoRqOl6Kq9qobUfjRC_/view?usp=sharing

select distinct b.customer_id, 
sum(a.list_price) over (partition by b.customer_id) sm 
max(a.list_price) over (partition by b.customer_id) m ,
min(a.list_price) over (partition by b.customer_id)  n,
count(a.list_price) over (partition by b.customer_id) cnt
from hw.transaction a left join hw.customer b on a.customer_id = b.custom r_id
where b.customer_id is not null order by sm desc, cnt desc

https://drive.google.com/file/d/1E-WNXfCilWCIhcNlvtmAY8C6Ont1iNM-/view?usp=sharing

#### ***Результат одинаков, если добавить distinct, в ином случае результат оконных функций - более 19 000 строк***

#### ***5-Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы.***

select b.first_name, b.last_name  from hw.customer b join (select customer_id, sum(list_price) as total_sales from hw.transaction 

group by customer_id) t on b.customer_id = t.customer_id 

where t.total_sales = (select min(total_sales) from (select sum(list_price) as total_sales from hw.transaction group by customer_id) min_sales)

https://drive.google.com/file/d/1rOKF1WkpW3ipb47SpW33pA6H49NzCvU5/view?usp=sharing

select b.first_name, b.last_name  from hw.customer b join (select customer_id, sum(list_price) as total_sales from hw.transaction 

group by customer_id) t on b.customer_id = t.customer_i

 where
t.total_sales = (select max(total_sales) from (select sum(list_price) as total_sales from hw.transaction group by customer_id) max_sales)

https://drive.google.com/file/d/1dx5fyLNMYllzaH_M4HhvSHMuh8Kz46wL/view?usp=sharing

#### ***6-Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.***
select customer_id, transaction_date, list_price

from (select customer_id, transaction_date, list_price ,

 row_number() over (partition by customer_id order by transaction_date) as row_n m
from hw.transaction) ranked_transactions where row_num = 1

https://drive.google.com/file/d/1i9vriVcGmA-PKVL1mMzQbdeuGhCvczcq/view?usp=sharing

#### ***если не джойнить таблицы, то в результате получится 3494 строки, в которой будет customer_id=5034, не существующий в таблице customer***

#### ***7-Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях)***

with TransactionIntervals as (select customer_id, 
transaction_date::date - lag(transaction_date::date) over (partition by customer_id order by transaction_date::date) as interval,
row_number() over (partition by customer_id order by transaction_date::date) as row_num
from hw.transaction) select c.first_name, c.last_name, c.job_title
from hw.customer c join (select customer_id from TransactionIntervals
where row_num > 1 group by customer_id having max(interval) = (select max(interval) from TransactionIntervals
where row_num > 1)) max_interval_customers ON c.customer_id = max_interval_customers.customer_id

https://drive.google.com/file/d/1FHQg7BUmem1eu7ZG1mh9IE-CJ1Y4UWYF/view?usp=sharing